Run this cell to initialize population synthesis

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from synthesize import *
from preprocess import *


config_folder = "../configs/Chicago"
n = 1000

person = process_pums_data(config_folder)
population_sample = synthesize_population(
    config_folder=config_folder, n_sample=n, min_age=18, max_age=65, random_state=2
)
ploc = puma_locations(config_folder)

Run this cell to initalize template generation

In [ ]:
smg = SystemMessageGenerator(config_folder, "SystemMessage.j2", verbose_debug=False)
year = 2019

def wrap_text(text, n):
    words = text.split()
    lines = []
    current_line = []

    for word in words:
        # Check if adding the next word would exceed the limit
        if sum(len(w) for w in current_line) + len(current_line) + len(word) <= n:
            current_line.append(word)
        else:
            # Join current line and start a new one
            lines.append(' '.join(current_line))
            current_line = [word]

    # Add last line if not empty
    if current_line:
        lines.append(' '.join(current_line))

    return '\n'.join(lines)

output = []
attribute_descriptions = get_attribute_descriptions(person)
for i, individual in population_sample.reset_index().iterrows():
    individual_attributes = attribute_decoder_dict(individual.to_dict(), person)
    system_message = smg.write_system_message(**individual_attributes, **attribute_descriptions, ploc=ploc, YEAR=year)
    print(wrap_text(system_message, 80)+"\n")

In [ ]:
import locale

locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
locale.currency(int(20000), grouping=True)[:-3]